In [ ]:
import xml.etree.ElementTree as et 
from gensim import corpora
from pprint import pprint 
from collections import defaultdict
import glob 

#Dictionary for all award ID's and Abstracts 
global awards
awards=dict()

#Array storage for Award IDs and Abstracts 
awardabs_ar=["Null"]
aID_ar=["Null"]

#Counts The number of items found with an Abstract vs without 
without = 0
T_with = 0 
grandtotal = 0

#Used as a condition for later statement 
x = 0

#Function Declaration 
def addIDandAbs(ID,abstract):
    for i in awards:
        #Checks for duplicates 
        if i == ID:
            print("Error")
            return
    #Store ID and Abstract
    awards[ID] = abstract
    awardabs.append(abstract)
    aID_ar.append(ID)
    
for items in glob.glob("201*/*.xml"): #Iterate through all xml files in the directory/file name given
    #Open files
    item = open(items)
    #Store Data
    str_data = item.read()
    #Convert data to text 
    data = et.fromstring(str_data)
    #store the current name of the institution 
    clemson = data.find("Award").find("Institution").find("Name").text
    #Check if the name matches Clemson University 
    if clemson=="Clemson University" :
        #Add award ID and Abstract to the dictionary awards
        #addIDandAbs(data.find("Award").find("AwardID").text,data.find("Award").find("AbstractNarration").text)
        ID = data.find("Award").find("AwardID").text
        abstract = data.find("Award").find("AbstractNarration").text
        awards[ID]=abstract
        #Keep count of the number of items found 
        T_with = T_with + 1
    else:
        #Optional Keep count of the number of items not found 
        without = without + 1
    #Optional Total Articles:
    grandtotal = grandtotal + 1
#Store Data into arrays from the library 
for i,v in awards.items():
    #Stores the first abstract and key into the first part of the array so append will see that the list is not empty.
    if x == 0:
        awardabs_ar[0]=v
        aID_ar[0]=i
    else:
        awardabs_ar.append(v)
        aID_ar.append(i)
    x = x + 1
#Optional print code for data in first part of array
#for l in range(T_with):
    #print("Award ID:\n"+aID_ar[l],"\n\nAward Abstract:\n"+awardabs_ar[l]+"\n")
file=open("cu_tigers.txt","w")    

awardabs_ar = map(lambda x: x+"\n", awardabs_ar)
file.writelines(awardabs_ar)


In [23]:
import logging
logging.basicConfig(format="%(asctime)s : %(levelname)s : %(message)s", level=logging.info)


from gensim import corpora,models,similarities
from pprint import pprint 
from collections import defaultdict
import os

file = open('cu_tigers.txt', 'r')
dictionary = file.readlines()

#removed word list    
r_words = 'side up end was all than ever despite often large has have been there know provide near two well some offer much higher low each within about such one uses over being approach its but if can which used or no other may who details it an few many thus must their more also these find at on via with would on find from due for a of the and to in not them is that will be as they will how understanding lay into by both are aspects give every new use this'
#remove common words and tokenize
stoplist = set(r_words.split())
texts = [[word for word in document.lower().split() if word not in stoplist]
          for document in dictionary]

# remove words that appear only once
frequency = defaultdict(int)
for text in texts:
    for token in text:
        frequency[token] += 1

texts = [[token for token in text if frequency[token] > 1]
          for text in texts]

dictionary2 = corpora.Dictionary(texts)

corpus = [dictionary2.doc2bow(text) for text in texts]

tfidf = models.TfidfModel(corpus) #initialize a model 
corpus_tfidf = tfidf[corpus] #transform corpus 

lsi = models.LsiModel(corpus_tfidf, id2word=dictionary2, num_topics=50)
corpus_lsi = lsi[corpus_tfidf]

lsi.print_topics(50)


[(0,
  '0.089*"engineering" + 0.085*"technology" + 0.083*"design" + 0.082*"privacy" + 0.081*"students" + 0.081*"power" + 0.080*"education" + 0.079*"virtual" + 0.078*"water" + 0.077*"systems"'),
 (1,
  '0.264*"conference" + 0.156*"talks" + -0.139*"detection" + -0.129*"power" + 0.118*"university" + -0.118*"technology" + 0.116*"geoscience" + 0.112*"speakers" + 0.111*"virtual" + 0.100*"graduate"'),
 (2,
  '-0.382*"privacy" + -0.148*"users" + 0.134*"conference" + -0.128*"data" + 0.120*"membrane" + -0.112*"network" + 0.099*"detection" + -0.098*"sharing" + -0.094*"provenance" + -0.093*"osns."'),
 (3,
  '0.350*"virtual" + 0.284*"geoscience" + -0.248*"conference" + 0.174*"field" + 0.159*"reality" + 0.144*"experiences" + -0.140*"talks" + 0.133*"earth" + -0.133*"privacy" + 0.127*"learning"'),
 (4,
  '-0.209*"virtual" + -0.164*"detection" + -0.163*"geoscience" + -0.153*"conference" + 0.143*"water" + -0.139*"technology" + -0.110*"privacy" + -0.109*"sensor" + -0.104*"voc" + -0.104*"field"'),
 (5,
  